In [1]:
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import time
import math
from datetime import timedelta

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

data = input_data.read_data_sets('data/MNIST',one_hot=True)

Extracting data/MNIST\train-images-idx3-ubyte.gz
Extracting data/MNIST\train-labels-idx1-ubyte.gz
Extracting data/MNIST\t10k-images-idx3-ubyte.gz
Extracting data/MNIST\t10k-labels-idx1-ubyte.gz


In [3]:
print('Size of')
print('Training Set\t\t{}'.format(len(data.train.labels)))
print('Test Set\t\t{}'.format(len(data.test.labels)))
print('Validation Set\t\t{}'.format(len(data.validation.labels)))

Size of
Training Set		55000
Test Set		10000
Validation Set		5000


In [4]:
data.test.cls = np.argmax(data.test.labels,axis=1)

In [5]:
##DIMENSIONS

img_size = 28

img_size_flat = img_size * img_size

img_shape = (img_size,img_size)

num_channels = 1

num_classes = 10

In [6]:
x = tf.placeholder(tf.float32,shape = [None,img_size_flat],name = 'x')

x_image = tf.reshape(x,[-1,img_size,img_size,num_channels])

y_true = tf.placeholder(tf.float32, shape = [None,num_classes],name = 'y_true')

y_true_cls = tf.argmax(y_true,axis=1)

In [7]:
def new_biases(length):
    return tf.Variable(tf.constant(0.05,shape=[length])) ### 0.05 is the constant value for biases

In [8]:
def new_weights(shape):  ## shape has to be passed as list
    return tf.Variable(tf.truncated_normal(shape,stddev=0.5))  ##Creation of weights is wrapped around a function

In [9]:
import prettytensor as pt
x_pretty = pt.wrap(x_image) ##Imp-- we are wrapping input as pretty tensor

In [10]:
with pt.defaults_scope(activation_fn=tf.nn.relu): ## By defaults-- now all activations are relu
    y_pred, loss = x_pretty.\  ## . means now x_pretty is i/p to conv2d
        conv2d(kernel=5, depth=16, name='layer_conv1').\   ## kernel means filter size and depth means no of filters
        max_pool(kernel=2, stride=2).\
        conv2d(kernel=5, depth=36, name='layer_conv2').\
        max_pool(kernel=2, stride=2).\
        flatten().\
        fully_connected(size=128, name='layer_fc1').\
        softmax_classifier(num_classes=num_classes, labels=y_true)

In [11]:
y_pred_cls = tf.argmax(y_pred,axis = 1) ##y_pred is is boolean now converted to int

In [13]:
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(loss)

correct_prediction = tf.equal(y_pred_cls,y_true_cls) ## Calculates vector of booleans

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) ## reduce_mean is average only

In [14]:
session = tf.Session()

session.run(tf.global_variables_initializer())  ## Initial
train_batch_size = 64


In [15]:
total_iterations = 0

def optimize(num_iterations):
    global total_iterations

    start_time = time.time()

    for i in range(total_iterations,
                   total_iterations + num_iterations):
        x_batch, y_true_batch = data.train.next_batch(train_batch_size)

        feed_dict_train = {x: x_batch,
                           y_true: y_true_batch}
        
        session.run(optimizer, feed_dict=feed_dict_train)
        
        if i % 100 == 0:
    
            acc = session.run(accuracy, feed_dict=feed_dict_train)
            msg = "Optimization Iteration: {0:>6}, Training Accuracy: {1:>6.1%}"
            print(msg.format(i + 1, acc))

    total_iterations += num_iterations
    end_time = time.time()
    time_dif = end_time - start_time
    print("Time usage: " + str(timedelta(seconds=int(round(time_dif)))))

In [16]:
def plot_example_errors(cls_pred, correct):
  
    incorrect = (correct == False)
    images = data.test.images[incorrect]
    
    cls_pred = cls_pred[incorrect]
    cls_true = data.test.cls[incorrect]
    
    plot_images(images=images[0:9],
                cls_true=cls_true[0:9],
                cls_pred=cls_pred[0:9])

In [17]:
def plot_confusion_matrix(cls_pred):
 
    cls_true = data.test.cls
    cm = confusion_matrix(y_true=cls_true,
                          y_pred=cls_pred)

    print(cm)
    plt.matshow(cm)
    plt.colorbar()
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, range(num_classes))
    plt.yticks(tick_marks, range(num_classes))
    plt.xlabel('Predicted')
    plt.ylabel('True')

    plt.show()

In [18]:
test_batch_size = 256

def print_test_accuracy(show_example_errors=False,
                        show_confusion_matrix=False):

    num_test = len(data.test.images)

    cls_pred = np.zeros(shape=num_test, dtype=np.int)
    i = 0

    while i < num_test:
        j = min(i + test_batch_size, num_test)

        images = data.test.images[i:j, :]

        labels = data.test.labels[i:j, :]
        feed_dict = {x: images,
                     y_true: labels}

        cls_pred[i:j] = session.run(y_pred_cls, feed_dict=feed_dict)

        i = j
        cls_true = data.test.cls
    correct = (cls_true == cls_pred)
    correct_sum = correct.sum()
    acc = float(correct_sum) / num_test

    msg = "Accuracy on Test-Set: {0:.1%} ({1} / {2})"
    print(msg.format(acc, correct_sum, num_test))
    if show_example_errors:
        print("Example errors:")
        plot_example_errors(cls_pred=cls_pred, correct=correct)

    if show_confusion_matrix:
        print("Confusion Matrix:")
        plot_confusion_matrix(cls_pred=cls_pred)

In [19]:
print_test_accuracy()

Accuracy on Test-Set: 15.1% (1510 / 10000)


In [20]:
optimize(num_iterations=1)

Optimization Iteration:      1, Training Accuracy:  20.3%
Time usage: 0:00:00


In [21]:
print_test_accuracy()

Accuracy on Test-Set: 18.2% (1817 / 10000)


In [22]:
optimize(num_iterations=99)

Time usage: 0:00:14


In [23]:
print_test_accuracy()

Accuracy on Test-Set: 88.4% (8841 / 10000)


In [24]:
optimize(num_iterations=900)

Optimization Iteration:    101, Training Accuracy:  92.2%
Optimization Iteration:    201, Training Accuracy:  96.9%
Optimization Iteration:    301, Training Accuracy: 100.0%
Optimization Iteration:    401, Training Accuracy:  93.8%
Optimization Iteration:    501, Training Accuracy:  93.8%
Optimization Iteration:    601, Training Accuracy:  92.2%
Optimization Iteration:    701, Training Accuracy:  96.9%
Optimization Iteration:    801, Training Accuracy:  92.2%
Optimization Iteration:    901, Training Accuracy:  96.9%
Time usage: 0:02:10


In [25]:
print_test_accuracy()

Accuracy on Test-Set: 96.4% (9638 / 10000)


In [26]:
optimize(num_iterations=9000)

Optimization Iteration:   1001, Training Accuracy:  96.9%
Optimization Iteration:   1101, Training Accuracy:  96.9%
Optimization Iteration:   1201, Training Accuracy:  96.9%
Optimization Iteration:   1301, Training Accuracy:  98.4%
Optimization Iteration:   1401, Training Accuracy:  96.9%
Optimization Iteration:   1501, Training Accuracy:  96.9%
Optimization Iteration:   1601, Training Accuracy:  98.4%
Optimization Iteration:   1701, Training Accuracy:  98.4%
Optimization Iteration:   1801, Training Accuracy:  98.4%
Optimization Iteration:   1901, Training Accuracy:  93.8%
Optimization Iteration:   2001, Training Accuracy:  98.4%
Optimization Iteration:   2101, Training Accuracy:  96.9%
Optimization Iteration:   2201, Training Accuracy: 100.0%
Optimization Iteration:   2301, Training Accuracy: 100.0%
Optimization Iteration:   2401, Training Accuracy:  96.9%
Optimization Iteration:   2501, Training Accuracy:  96.9%
Optimization Iteration:   2601, Training Accuracy:  98.4%
Optimization I

In [27]:
print_test_accuracy()   ###Accuracy is almost **90%**

Accuracy on Test-Set: 98.9% (9891 / 10000)


In [ ]:
###Problem ----> It is difficult to get weights from different layers